In [1]:
!pip install transformers datasets torch accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

In [2]:
import pandas as pd
from datasets import Dataset

# Load the dataset
data = pd.read_csv("/content/Dataset_m2_csv2.csv")

# Extract the lyrics column and remove NaNs
lyrics = data['text'].dropna().tolist()

# Create a text file for the lyrics
with open("lyrics.txt", "w") as file:
    for line in lyrics:
        file.write(line + "\n")

# Load the text file into a Hugging Face dataset
dataset = Dataset.from_text("lyrics.txt")


<ipython-input-2-95082b434ca4>:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/Dataset_m2_csv2.csv")


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True, truncation=True, max_length=512, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/64520 [00:00<?, ? examples/s]

In [4]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=500,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

# Fine-tune the model
trainer.train()


Step,Training Loss


KeyboardInterrupt: 

In [5]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load and preprocess the dataset
# Replace this with your dataset loading and preprocessing code
# Example:
# text_data = load_data("/path/to/dataset")
# train_data, val_data = train_test_split(text_data, test_size=0.1, random_state=42)

# Define training and validation datasets
train_dataset = ...  # Tokenized training dataset
val_dataset = ...    # Tokenized validation dataset

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    logging_steps=100,
    save_steps=100,
    eval_steps=500,
    logging_dir="./logs",
    report_to="none",
    max_steps=1000  # Specify the maximum number of training steps
)


# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


TypeError: object of type 'ellipsis' has no len()

In [7]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv("/content/Dataset_m2_csv2.csv")

# Extract the lyrics column and remove NaNs
lyrics = data['text'].dropna().tolist()

# Create a text file for the lyrics
with open("lyrics.txt", "w") as file:
    for line in lyrics:
        file.write(line + "\n")

# Load the text file into a Hugging Face dataset
dataset = Dataset.from_text("lyrics.txt")

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True, truncation=True, max_length=512, padding="max_length")

# Apply tokenization to the entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Print the tokenized dataset
print(tokenized_dataset)

# Split dataset into training and validation sets
train_size = int(0.9 * len(tokenized_dataset))
train_data, val_data = tokenized_dataset[:train_size], tokenized_dataset[train_size:]

# Print dataset sizes
print("Train dataset size:", len(train_data))
print("Validation dataset size:", len(val_data))


<ipython-input-7-076c4c687b31>:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/Dataset_m2_csv2.csv")


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/64520 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 64520
})
Train dataset size: 3
Validation dataset size: 3


In [8]:
import numpy as np

# Function to generate lyrics
def generate_lyrics(model, tokenizer, seed_text, max_length=100):
    input_ids = tokenizer.encode(seed_text, return_tensors='pt')
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
seed_text = "I have tried in vain"
generated_lyrics = generate_lyrics(model, tokenizer, seed_text, max_length=200)
print(generated_lyrics)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I have tried in vain to find a way to make the game more accessible to people who are not gamers. I have also tried to create a game that is accessible for everyone, but I am not sure that I can do that.

I am also not convinced that the developers of the games are aware of what they are doing. The game is not a "game" in the traditional sense of that word. It is a series of short stories that are told in a very simple way. There is no story, no plot, and no characters. Instead, the stories are about the characters and their relationships. They are stories about people and about their lives. In the end, they tell a story about how we can all be better people.
